# 分野データの取得

ダウンロードしたXMLファイルから、課題番号ごとの分野・分科・細目のデータ（field）を取得して、ローカルのデータベースに保存します。

In [ ]:
from lxml import etree
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import pickle
import glob
import sqlalchemy as sa
import sqlalchemy.ext.declarative
import pymysql
import re

## 研究課題ごとの研究分野（細目など）を抽出する関数

とりあえず、1985年度から2017年度まで、エラーなく回せるように作ってあります。

でも、もっとデータの構造をよく見たほうがよさそう。ここでは2014年度から2017年度（細目の動きがなかった期間）だけでやっています。

In [ ]:
def field(xmlfile):
    tree = etree.parse(xmlfile)
    nsmap = {"xml": "http://www.w3.org/XML/1998/namespace"}
    fieldlist = []
    for grantAward in tree.iterfind("grantAward"):
        awardnumber = grantAward.get("awardNumber")
        if grantAward.find("summary[@xml:lang='ja']/field", nsmap) is None:
            pass
        else:
            for field in grantAward.find("summary[@xml:lang='ja']", nsmap).iterfind("field", nsmap):
                field_sequence = field.get("sequence")
                field_path = field.get("path")
                field_niicode = field.get("niiCode")
                field_table = field.get("fieldTable")

                row = [
                    awardnumber,
                    field_sequence,
                    field_path,
                    field_niicode,
                    field_table,
                ]
                fieldlist.append(row)
            
    df = pd.DataFrame(fieldlist)
    
    if len(df.index) > 0:
        df.columns = [
            'awardnumber',
            'field_sequence',
            'field_path',
            'field_niicode',
            'field_table',
        ]
    
        pickledfile = 'pickledDF_field/' + re.search('[0-9]{4}_[0-9]+-[0-9]+.xml', xmlfile).group() + '.dump'
        df.to_pickle(pickledfile)

xmlファイルをパースして、pickleして保存

In [ ]:
import os
import shutil

def cleandir(dirname):
    if os.path.isdir(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

cleandir('pickledDF_field')

for i in tqdm(range(2014, 2018)):
    globdir = 'xml/' + str(i) + '*.xml'
    for xmlfile in tqdm(glob.glob(globdir)):
        field(xmlfile)

空のデータフレームを作って…

In [ ]:
columns = [
        'awardnumber',
        'field_sequence',
        'field_path',
        'field_niicode',
        'field_table',
]
df = pd.DataFrame(columns=columns)

pickleからデータフレームを復元、すべて結合

In [ ]:
for dump in tqdm(glob.glob('pickledDF_field/*.dump')):
    with open(dump, mode='rb') as f:
        df = pd.concat([df, pickle.load(f)])

カレントディレクトリにbeforeCleaningフォルダを作ります。

In [ ]:
import os

dirname = 'beforeCleaning'
if os.path.isdir(dirname) is False:
    os.mkdir(dirname)
    print('作成しました')
else:
    print('既に作成されています')    

連結したデータフレームをいったんpickleして保存

In [ ]:
df.to_pickle('beforeCleaning/parse_field.dump')

### データクリーニング

pickleからデータフレームを復元する

In [ ]:
with open('beforeCleaning/parse_field.dump', mode='rb') as f:
    df = pickle.load(f)

In [ ]:
df.info()

データ型を指定する

In [ ]:
df = df.fillna(0)
df = df.astype({
    'field_sequence': np.int64,
    'field_niicode': np.int64,
})
df.info()

細目選択数のカラムを作る。

2013-2017の若手Bでは複数細目を選択できたので、分野別に集計する際は分野ごとに按分する必要があるため。

In [ ]:
df['saimokusentakusuu'] = 1
df

In [ ]:
fukusuusaimokukadai_list = df[df['field_sequence'] == 2].awardnumber.unique()
len(fukusuusaimokukadai_list)

In [ ]:
df.loc[df['awardnumber'].isin(fukusuusaimokukadai_list), 'saimokusentakusuu'] = 2
df[df['saimokusentakusuu'] == 2]

awardnumberあたり8行になる。3565 * 8 == 28520 になっていたので特に問題なし。

awardnumberをインデックスに設定する

In [ ]:
df = df.set_index('awardnumber')

In [ ]:
df

ローカルのMariaDBに保存する

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('../settings/config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer

engine = create_engine(url, echo=True)

df.to_sql('grantaward_field', engine, if_exists='replace',
      dtype={
          'awardnumber': String(256),
          'field_sequence': Integer,
          'field_path': String(256),
          'field_niicode': Integer,
          'field_table': String(256),
      })

## おしまい

データがコミットされていれば終了。HeidiSQLなどで、上記データフレームの件数が登録されているか確認する。